<a href="https://colab.research.google.com/github/Urvashi-D/hacktoberfest/blob/urvashi_project1/project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Install libraries for natural language processing
!pip install spacy
!python -m spacy download en_core_web_sm

# Install SQLite for database querying (optional if you plan to test SQL queries)
import sqlite3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 60.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [8]:
import spacy

nlp = spacy.load('en_core_web_sm')
def break_down_query(query_text):
    parsed_query = nlp(query_text)
    word_list = [word.text for word in parsed_query]
    return word_list
def create_sql_query(query_text):
     if "orders" in query_text and "completed" in query_text and "July" in query_text:
        return "SELECT * FROM orders WHERE status = 'completed' AND month = 'July';"
    else:
        return "I don't understand the query."
user_query = "Get all completed orders in July"
parsed_words = break_down_query(user_query)
sql_statement = create_sql_query(user_query)
print("Words from query:", parsed_words)
print("Generated SQL:", sql_statement)


Words from query: ['Get', 'all', 'completed', 'orders', 'in', 'July']
Generated SQL: SELECT * FROM orders WHERE status = 'completed' AND month = 'July';
